In [1]:
# import required libraries
import numpy as np
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
import tensorflow as tf
tf.keras.utils.set_random_seed(1)

In [2]:
# loading files
from google.colab import files
uploaded = files.upload()

Saving BRCA.csv to BRCA.csv


In [3]:
cancer_type_df = pd.read_csv('BRCA.csv').set_index('Patient_ID')
cancer_type_df.head()

,Age,Gender,Protein1,Protein2,Protein3,Protein4,Tumour_Stage,Histology,ER status,PR status,HER2 status,Surgery_type,Date_of_Surgery,Date_of_Last_Visit,Patient_Status
Patient_ID,,,,,,,,,,,,,,,
TCGA-D8-A1XD,36.0,FEMALE,0.080353,0.42638,0.54715,0.273680,III,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Modified Radical Mastectomy,15-Jan-17,19-Jun-17,Alive
TCGA-EW-A1OX,43.0,FEMALE,-0.420320,0.57807,0.61447,-0.031505,II,Mucinous Carcinoma,Positive,Positive,Negative,Lumpectomy,26-Apr-17,09-Nov-18,Dead
TCGA-A8-A079,69.0,FEMALE,0.213980,1.31140,-0.32747,-0.234260,III,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Other,08-Sep-17,09-Jun-18,Alive
TCGA-D8-A1XR,56.0,FEMALE,0.345090,-0.21147,-0.19304,0.124270,II,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Modified Radical Mastectomy,25-Jan-17,12-Jul-17,Alive
TCGA-BH-A0BF,56.0,FEMALE,0.221550,1.90680,0.52045,-0.311990,II,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Other,06-May-17,27-Jun-19,Dead


In [4]:
cancer_type_df.dropna(inplace=True)

In [5]:
cancer_type_df['Histology'].value_counts()

Infiltrating Ductal Carcinoma     224
Infiltrating Lobular Carcinoma     81
Mucinous Carcinoma                 12
Name: Histology, dtype: int64

In [6]:
cancer_type_df.dtypes

Age                   float64
Gender                 object
Protein1              float64
Protein2              float64
Protein3              float64
Protein4              float64
Tumour_Stage           object
Histology              object
ER status              object
PR status              object
HER2 status            object
Surgery_type           object
Date_of_Surgery        object
Date_of_Last_Visit     object
Patient_Status         object
dtype: object

In [7]:
enc = OneHotEncoder(sparse = False)

In [8]:
cancer_type_df.drop(columns= ['Gender', 'Tumour_Stage', 'Surgery_type', "Date_of_Surgery", 'Date_of_Last_Visit', 'Patient_Status'], inplace=True)
cancer_type_df.head()

,Age,Protein1,Protein2,Protein3,Protein4,Histology,ER status,PR status,HER2 status
Patient_ID,,,,,,,,,
TCGA-D8-A1XD,36.0,0.080353,0.42638,0.54715,0.273680,Infiltrating Ductal Carcinoma,Positive,Positive,Negative
TCGA-EW-A1OX,43.0,-0.420320,0.57807,0.61447,-0.031505,Mucinous Carcinoma,Positive,Positive,Negative
TCGA-A8-A079,69.0,0.213980,1.31140,-0.32747,-0.234260,Infiltrating Ductal Carcinoma,Positive,Positive,Negative
TCGA-D8-A1XR,56.0,0.345090,-0.21147,-0.19304,0.124270,Infiltrating Ductal Carcinoma,Positive,Positive,Negative
TCGA-BH-A0BF,56.0,0.221550,1.90680,0.52045,-0.311990,Infiltrating Ductal Carcinoma,Positive,Positive,Negative


In [9]:
cancer_feat = cancer_type_df.copy().drop(columns = ['Histology'])
cancer_feat.head()

,Age,Protein1,Protein2,Protein3,Protein4,ER status,PR status,HER2 status
Patient_ID,,,,,,,,
TCGA-D8-A1XD,36.0,0.080353,0.42638,0.54715,0.273680,Positive,Positive,Negative
TCGA-EW-A1OX,43.0,-0.420320,0.57807,0.61447,-0.031505,Positive,Positive,Negative
TCGA-A8-A079,69.0,0.213980,1.31140,-0.32747,-0.234260,Positive,Positive,Negative
TCGA-D8-A1XR,56.0,0.345090,-0.21147,-0.19304,0.124270,Positive,Positive,Negative
TCGA-BH-A0BF,56.0,0.221550,1.90680,0.52045,-0.311990,Positive,Positive,Negative


In [10]:
object_variables = list(cancer_feat.dtypes[cancer_feat.dtypes == "object"].index)
object_variables

['ER status', 'PR status', 'HER2 status']

In [11]:
encoded_data = enc.fit_transform(cancer_feat[object_variables])

In [12]:
encoded_df = pd.DataFrame(encoded_data,columns= enc.get_feature_names_out(object_variables))
encoded_df.head()

,ER status_Positive,PR status_Positive,HER2 status_Negative,HER2 status_Positive
0,1.0,1.0,1.0,0.0
1,1.0,1.0,1.0,0.0
2,1.0,1.0,1.0,0.0
3,1.0,1.0,1.0,0.0
4,1.0,1.0,1.0,0.0


In [13]:
cancer_feat.drop(columns=object_variables, inplace=True)
cancer_feat.reset_index(inplace=True)
cancer_feat.head()

,Patient_ID,Age,Protein1,Protein2,Protein3,Protein4
0,TCGA-D8-A1XD,36.0,0.080353,0.42638,0.54715,0.273680
1,TCGA-EW-A1OX,43.0,-0.420320,0.57807,0.61447,-0.031505
2,TCGA-A8-A079,69.0,0.213980,1.31140,-0.32747,-0.234260
3,TCGA-D8-A1XR,56.0,0.345090,-0.21147,-0.19304,0.124270
4,TCGA-BH-A0BF,56.0,0.221550,1.90680,0.52045,-0.311990


In [14]:
X = pd.concat([cancer_feat, encoded_df], axis=1).set_index('Patient_ID')
X.head()

,Age,Protein1,Protein2,Protein3,Protein4,ER status_Positive,PR status_Positive,HER2 status_Negative,HER2 status_Positive
Patient_ID,,,,,,,,,
TCGA-D8-A1XD,36.0,0.080353,0.42638,0.54715,0.273680,1.0,1.0,1.0,0.0
TCGA-EW-A1OX,43.0,-0.420320,0.57807,0.61447,-0.031505,1.0,1.0,1.0,0.0
TCGA-A8-A079,69.0,0.213980,1.31140,-0.32747,-0.234260,1.0,1.0,1.0,0.0
TCGA-D8-A1XR,56.0,0.345090,-0.21147,-0.19304,0.124270,1.0,1.0,1.0,0.0
TCGA-BH-A0BF,56.0,0.221550,1.90680,0.52045,-0.311990,1.0,1.0,1.0,0.0


In [15]:
y = cancer_type_df['Histology']
y.value_counts()

Infiltrating Ductal Carcinoma     224
Infiltrating Lobular Carcinoma     81
Mucinous Carcinoma                 12
Name: Histology, dtype: int64

In [16]:
number_of_classes = len(list(y.unique()))
number_of_predictors = len(X.columns)
print(number_of_classes)
print(number_of_predictors)

3
9


In [17]:
encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)
encoded_y

array([0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 2, 1, 2, 0, 0, 0, 0, 0, 0,
       0, 2, 2, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 2, 0, 0, 2, 0, 0,
       0, 1, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1,
       1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1,

In [18]:
from keras.utils import to_categorical
y_categorical = to_categorical(encoded_y, num_classes=number_of_classes)
y_categorical

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, random_state = 1)

In [20]:
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [21]:
nn = Sequential()

In [22]:
nn.add(Dense(25, input_dim = number_of_predictors, activation = 'relu'))
nn.add(Dense(25, activation = 'relu'))
nn.add(Dense(25, activation = 'tanh'))
nn.add(Dense(25, activation = 'tanh'))
nn.add(Dropout(.1))
nn.add(Dense(number_of_classes, activation='softmax'))

In [23]:
nn.compile(loss="categorical_crossentropy",
              optimizer= "adam",
              metrics=['categorical_accuracy'])

In [24]:
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 25)                250       
                                                                 
 dense_1 (Dense)             (None, 25)                650       
                                                                 
 dense_2 (Dense)             (None, 25)                650       
                                                                 
 dense_3 (Dense)             (None, 25)                650       
                                                                 
 dropout (Dropout)           (None, 25)                0         
                                                                 
 dense_4 (Dense)             (None, 3)                 78        
                                                                 
Total params: 2278 (8.90 KB)
Trainable params: 2278 (8.9

In [25]:
number_of_epochs = 200
nn.fit(X_train, y_train,
       epochs = number_of_epochs,
       shuffle = True)

Epoch 1/200
8/8 [==============================] - 4s 13ms/step - loss: 1.2102 - categorical_accuracy: 0.3629
Epoch 2/200
8/8 [==============================] - 0s 8ms/step - loss: 0.8329 - categorical_accuracy: 0.6709
Epoch 3/200
8/8 [==============================] - 0s 10ms/step - loss: 0.7601 - categorical_accuracy: 0.6793
Epoch 4/200
8/8 [==============================] - 0s 7ms/step - loss: 0.7932 - categorical_accuracy: 0.6371
Epoch 5/200
8/8 [==============================] - 0s 7ms/step - loss: 0.7706 - categorical_accuracy: 0.6624
Epoch 6/200
8/8 [==============================] - 0s 6ms/step - loss: 0.7956 - categorical_accuracy: 0.6878
Epoch 7/200
8/8 [==============================] - 0s 7ms/step - loss: 0.8041 - categorical_accuracy: 0.6793
Epoch 8/200
8/8 [==============================] - 0s 9ms/step - loss: 0.7728 - categorical_accuracy: 0.6751
Epoch 9/200
8/8 [==============================] - 0s 7ms/step - loss: 0.7779 - categorical_accuracy: 0.6540
Epoch 10/200
8/8 

In [26]:
nn.evaluate(X_test,y_test, verbose=2)

3/3 - 0s - loss: 0.6632 - categorical_accuracy: 0.7625 - 205ms/epoch - 68ms/step


[0.6632073521614075, 0.762499988079071]

In [27]:
predictions = nn.predict(X_test)
predictions

3/3 [==============================] - 0s 4ms/step


array([[0.6370824 , 0.3340461 , 0.02887153],
       [0.46477777, 0.46497172, 0.07025039],
       [0.4743316 , 0.47168705, 0.05398122],
       [0.7635673 , 0.18849485, 0.04793787],
       [0.69796   , 0.27409166, 0.02794821],
       [0.73609257, 0.23485464, 0.02905271],
       [0.65945286, 0.3080702 , 0.03247695],
       [0.69991255, 0.2444613 , 0.05562614],
       [0.6465089 , 0.32883775, 0.02465336],
       [0.64982617, 0.31438592, 0.03578782],
       [0.5751855 , 0.33563694, 0.08917751],
       [0.5901837 , 0.3731342 , 0.03668198],
       [0.6822127 , 0.26171795, 0.05606934],
       [0.68880206, 0.28676036, 0.02443758],
       [0.62718993, 0.32076427, 0.05204578],
       [0.7431955 , 0.22160016, 0.03520435],
       [0.6565109 , 0.22942634, 0.11406267],
       [0.75866264, 0.1877283 , 0.0536091 ],
       [0.6571154 , 0.29472262, 0.04816204],
       [0.72113085, 0.2238817 , 0.0549874 ],
       [0.7695076 , 0.17195451, 0.05853795],
       [0.7551307 , 0.17880736, 0.06606183],
       [0.

In [28]:
most_likely = np.argmax(predictions, axis=1)
most_likely

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [29]:
most_likely = encoder.inverse_transform((most_likely))
most_likely

array(['Infiltrating Ductal Carcinoma', 'Infiltrating Lobular Carcinoma',
       'Infiltrating Ductal Carcinoma', 'Infiltrating Ductal Carcinoma',
       'Infiltrating Ductal Carcinoma', 'Infiltrating Ductal Carcinoma',
       'Infiltrating Ductal Carcinoma', 'Infiltrating Ductal Carcinoma',
       'Infiltrating Ductal Carcinoma', 'Infiltrating Ductal Carcinoma',
       'Infiltrating Ductal Carcinoma', 'Infiltrating Ductal Carcinoma',
       'Infiltrating Ductal Carcinoma', 'Infiltrating Ductal Carcinoma',
       'Infiltrating Ductal Carcinoma', 'Infiltrating Ductal Carcinoma',
       'Infiltrating Ductal Carcinoma', 'Infiltrating Ductal Carcinoma',
       'Infiltrating Ductal Carcinoma', 'Infiltrating Ductal Carcinoma',
       'Infiltrating Ductal Carcinoma', 'Infiltrating Ductal Carcinoma',
       'Infiltrating Ductal Carcinoma', 'Infiltrating Ductal Carcinoma',
       'Infiltrating Ductal Carcinoma', 'Infiltrating Ductal Carcinoma',
       'Infiltrating Ductal Carcinoma', 'Infiltrat

In [30]:
pd.DataFrame(most_likely).value_counts()

Infiltrating Ductal Carcinoma     78
Infiltrating Lobular Carcinoma     2
dtype: int64